# Chicago Crime Rate Interactive Dashboard

In [1]:
import pandas as pd
import numpy as np

import datetime
import calendar

from ipywidgets import widgets, Accordion, Button, Dropdown, HBox, Output, Tab, VBox, Layout, GridBox
import io

import folium
from folium import plugins

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("Crimes_-_2001_to_Present.csv")

df.Latitude = df.Latitude.fillna(df.Latitude.mean(skipna=True))
df.Longitude = df.Longitude.fillna(df.Longitude.mean(skipna=True))



In [3]:
df2 = df.drop(columns=['ID', 'Case Number', 'IUCR', 'Block', 'FBI Code', 'X Coordinate', 
                       'Y Coordinate', 'Updated On', 'Location', 'Beat','District','Ward', 'Community Area'])

In [4]:
df2 = df2.rename(columns={'Date':'Datetime'})
df2.Datetime = pd.to_datetime(df2.Datetime, format='%m/%d/%Y %I:%M:%S %p')
df2['Date'] = df2['Datetime'].dt.date
df2['Time'] = df2['Datetime'].dt.time 

In [5]:
# values_loc_desc = list(df2['Location Description'].dropna().unique())
# values_pri_desc = list(df2['Primary Type'].dropna().unique())
# values_sec_desc = list(df2['Description'].dropna().unique())


values_loc_desc = list(df2['Location Description'].value_counts()[:20].index)
values_pri_desc = list(df2['Primary Type'].value_counts()[:20].index)
values_sec_desc = list(df2['Description'].value_counts()[:20].index)

center = (df2.Latitude.mean(skipna=True), df2.Longitude.mean(skipna=True))

In [6]:
def get_df(from_d, to_d, loc_list, pri_list, sec_list, arrest_val, domestic_val):
    table = df2[(df2['Date']>datetime.date(from_d.year,from_d.month,from_d.day)) & (df2['Date']<datetime.date(to_d.year,to_d.month,to_d.day))]
    # Arrest and Domestic
    if arrest_val == 'True':
        table = table[table['Arrest']==True]
    if arrest_val == 'False':
        table = table[table['Arrest']==False]
    if domestic_val == 'True':
        table = table[table['Domestic']==True]
    if domestic_val == 'False':
        table = table[table['Domestic']==False]

    # Location, Primary, Secondary
    table = table[table['Location Description'].isin(loc_list)]
    table = table[table['Primary Type'].isin(pri_list)]
    table = table[table['Description'].isin(sec_list)]
    
    return table

In [7]:
date_from = widgets.DatePicker(description='From', disabled=False, value = datetime.date(2015, 4, 1))
date_to = widgets.DatePicker(description='To',disabled=False, value = datetime.date(2019, 6, 1))

arrest = widgets.RadioButtons(
                    options=[('Yes','True'), ('No','False'),('None','none')],
                    value='none',
                    description='Arrest',
                    disabled=False
                )
domestic = widgets.RadioButtons(
                    options=[('Yes','True'), ('No','False'),('None','none')],
                    value='none',
                    description='Domestic',
                    disabled=False
                )

In [8]:
loc_desc = widgets.SelectMultiple(
                options = values_loc_desc,
                value=['STREET'],
                rows=5,
                description='Location',
                disabled=False
            )
pri_desc = widgets.SelectMultiple(
                options = values_pri_desc,
                value=['BATTERY'],
                rows=5,
                description='Primary Type',
                disabled=False
            )
sec_desc = widgets.SelectMultiple(
                options = values_sec_desc,
                value=['SIMPLE'],
                rows=5,
                description='Secondary Type',
                disabled=False
            )

In [9]:
accordion = widgets.Accordion(children=[loc_desc, pri_desc, sec_desc])
accordion.set_title(0, 'Location Description')
accordion.set_title(1, 'Crime-Primary Type')
accordion.set_title(2, 'Crime-Secondary Type')

In [10]:
control = VBox([date_from, date_to, arrest, domestic, accordion])

In [11]:
# Heat Map
def heat_map(date_from, date_to, loc_desc, pri_desc, sec_desc, arrest, domestic):    
    table = get_df(date_from, date_to, loc_desc, pri_desc, sec_desc, arrest, domestic)
    
    Location = table[['Latitude', 'Longitude']].to_numpy()
    m = folium.Map(location=list(center), tiles='cartodb positron')
    m = m.add_child(plugins.HeatMap(Location, radius=12))
    display(m)
    

In [12]:
def circle_map(date_from, date_to, loc_desc, pri_desc, sec_desc, arrest, domestic):
    table = get_df(date_from, date_to, loc_desc, pri_desc, sec_desc, arrest, domestic)
    
    Location = table[['Latitude', 'Longitude']].to_numpy()
    m = folium.Map(location=list(center),tiles='CartoDB dark_matter')
    plugins.FastMarkerCluster(data=Location).add_to(m)
    folium.LayerControl().add_to(m)
    
    for index, row in table.iterrows():
        popup_text = "Date: {}<br> Time: {}<br> Location Description: {}<br> Primary Type: {}<br> Secondary Type: {}"
        popup_text = popup_text.format(
                      row['Date'].strftime("%b %d, %Y"), row['Time'].strftime('%I:%M:%S %p'),
                      row['Location Description'], row['Primary Type'], row['Description']
                      )
        folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
                            radius= 3,
                            color="#007849",
                            popup=folium.Popup(popup_text, max_width=500,min_width=100),
                            fill=False).add_to(m)
    
    display(m)
    

In [13]:
# out = Output()
map1 = widgets.interactive_output(heat_map, 
                                  {'date_from':date_from, 'date_to':date_to, 
                                   'loc_desc':loc_desc, 'pri_desc':pri_desc, 
                                   'sec_desc':sec_desc, 'arrest':arrest, 
                                   'domestic':domestic})
map2 = widgets.interactive_output(circle_map, 
                                  {'date_from':date_from, 'date_to':date_to, 
                                   'loc_desc':loc_desc, 'pri_desc':pri_desc, 
                                   'sec_desc':sec_desc, 'arrest':arrest, 
                                   'domestic':domestic})

maps = Tab(layout=Layout(height='100%'))
maps.children = [map1, map2]
# maps.set_title(0,'Debug Information')
maps.set_title(0, 'Heat Map')
maps.set_title(1, 'Fast Marker Cluster')

In [14]:
GridBox(children = [control, maps],
       layout=Layout(width='100%', 
                    grid_template_columns='30% 70%',
                    grid_template_areas='''
                    "control maps"
                    '''))

GridBox(children=(VBox(children=(DatePicker(value=datetime.date(2015, 4, 1), description='From'), DatePicker(v…